# Trabajo Fin de Máster  
## Análisis de sentimientos en noticias financieras mediante Aprendizaje Automático y Procesamiento del Lenguaje Natural  


*   ### Autor: Pablo Bayón Gala
*   ### Universidad: Universidad Internacional de La Rioja
*   ### Máster: Máster Universitario en Inteligencia Artificial  
*   ### Fecha: Septiembre 2025


---


© 2025 Pablo Bayón Gala.

Este cuaderno contiene el código y los experimentos realizados para el desarrollo del Trabajo Fin de Máster, en el que se implementan y comparan distintos modelos de Procesamiento del Lenguaje Natural aplicados al análisis de sentimiento en textos financieros.

Se distribuye bajo la **GNU General Public License v3.0 (GPL v3)**.

Para más detalles sobre la licencia: [https://www.gnu.org/licenses/gpl-3.0.html](https://www.gnu.org/licenses/gpl-3.0.html)


# Preprocesamiento de los datos

El preprocesamiento de datos siguiente lo voy a estructurar en forma de pipeline diferenciando dos tipos:

1. Por un lado, los tres modelos basados en arquitectura Transformers (FinBERT, DistilRoBERTa con fine-tuning y BERTweet) ya incluyen un tokenizador preentrenado junto con el modelo. Dicho tokenizador espera el texto en bruto (con mayúsculas, tildes, hashtags, etc.) porque su preentrenamiento se realizó con texto real provenientes de noticias / tweets. Para estos modelos, se implementa una limpieza ligera para no alterar el lenguaje natural. Su flujo de limpieza consiste en los siguientes pasos:
  * Eliminación de duplicados
  *	Eliminación de tweets vacíos
  *	Eliminación de URLs

2. Por otro lado, el modelo basado en diccionarios y reglas (VADER) requiere un preprocesamiento más profundo. Su flujo de limpieza se basa en los siguientes pasos:
  *	Eliminación de duplicados
  *	Eliminación de tweets vacíos
  *	Eliminación de URLs
  *	Eliminación de menciones (@usuario)
  *	Eliminación de hashtags
  *	Eliminación de espacios adicionales

In [ ]:
""" Pipeline de preprocesamiento para modelos basados en Transformers """
def limpieza_transformers(texto):

  # Eliminamos URLs
  texto = re.sub(r"(http[s]?://\S+)|(www\.\S+)", "", texto)

  return texto

""" Pipeline de preprocesamiento para VADER """
def limpieza_vader(texto):

  # Eliminamos menciones
  texto = re.sub('@[^\s]+','',texto)
  # Eliminamos hashtags (manteniendo la palabra del hashtag)
  texto = re.sub(r'#','',texto)
  # Eliminamos URLs
  texto = re.sub(r"(http[s]?://\S+)|(www\.\S+)", "", texto)
  # Reemplazamos múltiples espacios seguidos por un solo espacio
  texto = re.sub(r'\s+', ' ', texto, flags=re.I)

  return texto

# Evaluación de mis modelos PLN a partir de los conjuntos de datos etiquetados

## Dataset sobre titulares de noticias financieras
**Labeled Stock News Headlines**

URL: https://www.kaggle.com/datasets/johoetter/labeled-stock-news-headlines

In [ ]:
import pandas as pd

# Cargo el dataset
df_Stock_News_Headlines = pd.read_csv('stock_news.csv')

df_Stock_News_Headlines.head()

In [ ]:
df_Stock_News_Headlines.shape

In [ ]:
df_Stock_News_Headlines.info()

In [ ]:
import matplotlib.pyplot as plt

# Datos
counts = df_Stock_News_Headlines['label'].value_counts()
labels = counts.index

# Paleta de colores agradable (pastel o profesional)
colors = plt.cm.Pastel1(range(len(labels)))

# Gráfico
fig, ax = plt.subplots(figsize=(6, 6))
wedges, texts, autotexts = ax.pie(
    counts,
    labels=labels,
    autopct='%1.1f%%',
    startangle=90,
    colors=colors,
    textprops={'fontsize': 12, 'color': 'black'}
)

# Mejorar visibilidad de porcentajes
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_fontweight('bold')

# Título con formato profesional
ax.set_title(
    "Distribución de sentimiento en dataset original",
    fontsize=12,
    fontweight='bold',
    pad=20
)

# Mantener el pastel como círculo perfecto
ax.axis('equal')

# Quitar el borde de las etiquetas para un look limpio
plt.setp(texts, fontweight='medium')

plt.show()

In [ ]:
# BALANCEAMOS EL DATASET
# Comprobamos distribución inicial
print("Distribución original:")
print(df_Stock_News_Headlines['label'].value_counts(normalize=True) * 100)

# Determinamos tamaño de la clase minoritaria (Negative)
min_class = "Negative"
min_count = df_Stock_News_Headlines[df_Stock_News_Headlines['label'] == min_class].shape[0]
print(f"\nTamaño de la clase minoritaria ({min_class}): {min_count}")

# Tomamos todos los textos de la clase minoritaria
df_min = df_Stock_News_Headlines[df_Stock_News_Headlines['label'] == min_class]

# Submuestreamos aleatoriamente las otras clases para igualar tamaño
balanced_parts = [df_min]  # empezamos con la clase minoritaria

for label in ["Positive", "Neutral"]:
    df_label = df_Stock_News_Headlines[df_Stock_News_Headlines['label'] == label].sample(
        n=min_count,
        random_state=42
    )
    balanced_parts.append(df_label)

# Combinamos en un dataset balanceado
df_Stock_News_Headlines_balanceado = pd.concat(balanced_parts).sample(frac=1, random_state=42).reset_index(drop=True)

# Comprobamos distribución final
print("\nDistribución balanceada:")
print(df_Stock_News_Headlines_balanceado['label'].value_counts())

# Guardamos el dataset balanceado
df_Stock_News_Headlines_balanceado.to_excel("stock_news_balanced.xlsx", index=False)

print("\nDataset balanceado guardado correctamente.")

In [ ]:
df_Stock_News_Headlines_balanceado.shape

### 1. Modelo FinBERT

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

# Cargar FinBERT desde Hugging Face
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Crear el pipeline de FinBERT
finbert = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=False, truncation=True)

# Aplicar FinBERT a cada titular
df_Stock_News_Headlines_balanceado['FinBERT_sentiment'] = df_Stock_News_Headlines_balanceado['headline'].apply(lambda x: finbert(x)[0]['label'])
df_Stock_News_Headlines_balanceado['confidence'] = df_Stock_News_Headlines_balanceado['headline'].apply(lambda x: finbert(x)[0]['score'])

# Mostrar resultados
print(df_Stock_News_Headlines_balanceado)
df_Stock_News_Headlines_balanceado.to_excel("LSNH_balanceado_FinBERT.xlsx", index=False)

In [ ]:
# 1. Vemos la arquitectura de alto nivel del modelo
print(model)

In [ ]:
# 2. Accedemos a la configuración del modelo
print(model.config)

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Asegurarse de que las etiquetas estén en el mismo formato (ej. capitalización)
df_Stock_News_Headlines["label"] = df_Stock_News_Headlines["label"].str.lower().str.strip()
df_Stock_News_Headlines["sentiment"] = df_Stock_News_Headlines["sentiment"].str.lower().str.strip()

# Calcular matriz de confusión
cm = confusion_matrix(df_Stock_News_Headlines["label"], df_Stock_News_Headlines["sentiment"], labels=["positive", "negative", "neutral"])

# Mostrar como DataFrame
cm_df_Stock_News_Headlines = pd.DataFrame(cm, index=["True_Positive", "True_Negative", "True_Neutral"],
                        columns=["Pred_Positive", "Pred_Negative", "Pred_Neutral"])

print("Matriz de confusión:")
print(cm_df_Stock_News_Headlines)

# Reporte de métricas
print("\nReporte de clasificación:")
print(classification_report(df_Stock_News_Headlines["label"], df_Stock_News_Headlines["sentiment"], digits=3))

# Visualización con heatmap
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Pred_Positive", "Pred_Negative", "Pred_Neutral"],
            yticklabels=["True_Positive", "True_Negative", "True_Neutral"])
plt.xlabel("PREDICCIONES")
plt.ylabel("VALORES REALES")
plt.title("Matriz de Confusión (Dataset Noticias - Balanceado)")
plt.show()

### 2. Modelo DistilRoBERTa + Fine-tuning

In [ ]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Cargar el modelo desde Hugging Face
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Creamos el pipeline con el modelo
sentiment_model = pipeline(
    "sentiment-analysis",
    model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
)

# Función para analizar sentimiento
def analyze_sentiment(text):
    result = sentiment_model(text)[0]
    return pd.Series([result['label'], result['score']])

# Aplicar análisis a los titulares
df_Stock_News_Headlines_balanceado[['DistilRoBERTa_sentiment', 'confidence']] = df_Stock_News_Headlines_balanceado['headline'].apply(analyze_sentiment)

print(df_Stock_News_Headlines_balanceado)
df_Stock_News_Headlines_balanceado.to_excel("LSNH_balanceado_DistilRoBERTa.xlsx", index=False)

In [ ]:
# 1. Vemos la arquitectura de alto nivel del modelo
print(model)

In [ ]:
# 2. Accedemos a la configuración del modelo
print(model.config)

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Asegurarse de que las etiquetas estén en el mismo formato (ej. capitalización)
df_Stock_News_Headlines_balanceado["label"] = df_Stock_News_Headlines_balanceado["label"].str.lower().str.strip()
df_Stock_News_Headlines_balanceado["DistilRoBERTa_sentiment"] = df_Stock_News_Headlines_balanceado["DistilRoBERTa_sentiment"].str.lower().str.strip()

# Calcular matriz de confusión
cm = confusion_matrix(df_Stock_News_Headlines_balanceado["label"], df_Stock_News_Headlines_balanceado["DistilRoBERTa_sentiment"], labels=["positive", "negative", "neutral"])

# Mostrar como DataFrame
cm_df_Stock_News_Headlines_balanceado = pd.DataFrame(cm, index=["True_Positive", "True_Negative", "True_Neutral"],
                        columns=["Pred_Positive", "Pred_Negative", "Pred_Neutral"])

print("Matriz de confusión:")
print(cm_df_Stock_News_Headlines_balanceado)

# Reporte de métricas
print("\nReporte de clasificación:")
print(classification_report(df_Stock_News_Headlines_balanceado["label"], df_Stock_News_Headlines_balanceado["DistilRoBERTa_sentiment"], digits=3))

# Visualización con heatmap
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Pred_Positive", "Pred_Negative", "Pred_Neutral"],
            yticklabels=["True_Positive", "True_Negative", "True_Neutral"])
plt.xlabel("PREDICCIONES")
plt.ylabel("VALORES REALES")
plt.title("Matriz de Confusión (Dataset Noticias - Balanceado)")
plt.show()

### 3. Modelo BERTweet

In [ ]:
# Importar el modelo y tokenizer de BERTweet
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
from tqdm import tqdm

# Modelo fine-tuned para análisis de sentimiento
MODEL_NAME = "finiteautomata/bertweet-base-sentiment-analysis" # Este modelo ya está afinado específicamente para clasificación de sentimiento en tweets
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# Creamos una función para predecir sentimiento
def predict_sentiment(tweet: str) -> str:
    inputs = tokenizer(tweet, return_tensors="pt", truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = F.softmax(logits, dim=1)
        label_id = torch.argmax(probs, dim=1).item()

    labels = model.config.id2label
    return labels[label_id]

# Lo aplico a mis tweets en formato original (sin preprocesado) porque el modelo de Hugging Face está entrenado para interpretar todo (emojis, menciones, etc.) como señales de sentimiento
tqdm.pandas() # Para mostrar una barra de progreso
# Convierto los valores a cadenas y reemplazo los NaN por texto vacío
df_Stock_News_Headlines_balanceado['headline'] = df_Stock_News_Headlines_balanceado['headline'].fillna('').astype(str)
# Creamos una nueva columna llamada bertweet_sentiment con etiquetas como NEG, NEU, POS llamando a la función anterior
df_Stock_News_Headlines_balanceado['BERTweet_sentiment'] = df_Stock_News_Headlines_balanceado['headline'].progress_apply(predict_sentiment)

# Guardamos los resultados
df_Stock_News_Headlines_balanceado.to_excel("LSNH_balanceado_BERTweet.xlsx", index=False)
df_Stock_News_Headlines_balanceado.head()

In [ ]:
# 1. Vemos la arquitectura de alto nivel del modelo
print(model)

In [ ]:
# 2. Accedemos a la configuración del modelo
print(model.config)

In [ ]:
# Tengo que mapear la etiqueta de sentiminto generada por mi modelo
def map_labels(label):
    if label == "POS":
        return "POSITIVE"
    elif label == "NEG":
        return "NEGATIVE"
    else:
        return "NEUTRAL"

df_Stock_News_Headlines_balanceado["BERTweet_sentiment_mapped"] = df_Stock_News_Headlines_balanceado["BERTweet_sentiment"].apply(map_labels)

df_Stock_News_Headlines_balanceado.head()

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Asegurarse de que las etiquetas estén en el mismo formato (ej. capitalización)
df_Stock_News_Headlines_balanceado["label"] = df_Stock_News_Headlines_balanceado["label"].str.lower().str.strip()
df_Stock_News_Headlines_balanceado["BERTweet_sentiment_mapped"] = df_Stock_News_Headlines_balanceado["BERTweet_sentiment_mapped"].str.lower().str.strip()

# Calcular matriz de confusión
cm = confusion_matrix(df_Stock_News_Headlines_balanceado["label"], df_Stock_News_Headlines_balanceado["BERTweet_sentiment_mapped"], labels=["positive", "negative", "neutral"])

# Mostrar como DataFrame
cm_df_Stock_News_Headlines_balanceado = pd.DataFrame(cm, index=["True_Positive", "True_Negative", "True_Neutral"],
                        columns=["Pred_Positive", "Pred_Negative", "Pred_Neutral"])

print("Matriz de confusión:")
print(cm_df_Stock_News_Headlines_balanceado)

# Reporte de métricas
print("\nReporte de clasificación:")
print(classification_report(df_Stock_News_Headlines_balanceado["label"], df_Stock_News_Headlines_balanceado["BERTweet_sentiment_mapped"], digits=3))

# Visualización con heatmap
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Pred_Positive", "Pred_Negative", "Pred_Neutral"],
            yticklabels=["True_Positive", "True_Negative", "True_Neutral"])
plt.xlabel("PREDICCIONES")
plt.ylabel("VALORES REALES")
plt.title("Matriz de Confusión (Dataset Noticias - Balanceado)")
plt.show()

### 4. Modelo VADER

In [ ]:
# Instalo librerías necesarias
!pip install pyspellchecker
!pip install scattertext
!pip install nltk
!pip install -U kaleido

In [ ]:
# Import Data Preprocessing and Wrangling libraries
import re
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from datetime import datetime
import dateutil.parser

# Import NLP Libraries
import nltk
from spellchecker import SpellChecker
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

# Import Visualization Libraries
import plotly.offline as pyo
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import seaborn as sns
import scattertext as st
from IPython.display import IFrame
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import random

# Downloading periphrals
nltk.download('vader_lexicon')
nltk.download('stopwords')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Inicializo herramientas

# Para visualizaciones con seaborn
sns.set_style('darkgrid')

# Análisis de sentimiento con VADER
sia = SIA()

# Corrector ortográfico (puede ayudar antes del análisis de sentimiento)
spell = SpellChecker()

# Para mostrar gráficos Plotly en notebooks
pyo.init_notebook_mode()

In [ ]:
""" Aplicamos pipeline de preprocesamiento para VADER (limpieza más profunda)"""

# Se crea una copia del DataFrame original df_tweets para no modificarlo directamente
data = df_Stock_News_Headlines_balanceado.copy()
# Se añade una columna original_tweet con el texto sin procesar a modo de backup para comparar
data['original_headline'] = df_Stock_News_Headlines_balanceado['headline']
# Reemplaza los valores nulos (NaN) en la columna "text" por cadenas vacías '' y convierte todo el contenido de la columna "text" a tipo string.
# Esto sirve para evitar que el modelo falle al encontrarse con un NaN y para garantizar que todo se maneje como string.
data['headline'] = data['headline'].fillna('').astype(str)

# Aplica la función de limpieza a la columna "text"
data['headline'] = data['headline'].apply(limpieza_vader)
# Eliminamos duplicados
data.drop_duplicates(subset=["headline"], inplace=True)
# Reseteamos el índice por si se ha eliminado alguna fila
data.reset_index(drop=True, inplace=True)

data.head()

In [ ]:
import textwrap

# Ancho máximo de cada línea (ejemplo: 80 caracteres)
width = 80

print("="*80)
print("headline ANTES de preprocesamiento:")
print("="*80)
print(textwrap.fill(data.original_headline[0], width=width))  # Aquí hace el salto de línea
print("\n" + "="*80)
#print(data.original_tweet[0])
print("headline DESPUÉS de preprocesamiento:")
print(80*"=")
print(textwrap.fill(data.headline[0], width=width))  # También aquí
#print(data.text[0])

In [ ]:
# Esta función convierte un valor de sentimiento (entre -1 y 1, generado por VADER) en una etiqueta categórica
def label_sentiment(x:float):
    if x < -0.05 : return 'negative'  # negative si es muy bajo
    if x > 0.05 : return 'positive'   # positive si es alto
    return 'neutral'                  # neutral si está entre ambos umbrales

# EXTRACCIÓN DE CARACTERÍSTICAS del texto
# Extrae todas las palabras de cada tweet usando regex, eliminando puntuación.
data['words'] = data.headline.apply(lambda x:re.findall(r'\w+', x ))
# Usa SpellChecker para detectar palabras mal escritas
data['errors'] = data.words.apply(spell.unknown)
# Cuenta cuántos errores ortográficos y cuántas palabras hay por tweet
data['errors_count'] = data.errors.apply(len)
data['words_count'] = data.words.apply(len)
# Longitud del tweet (en caracteres)
data['sentence_length'] = data.headline.apply(len)

# Análisis de sentimiento para cada tweet
# Aplica VADER (SentimentIntensityAnalyzer = SIA) a cada tweet para obtener el sentimiento compuesto (compound score), que va de -1 (negativo) a 1 (positivo).
data['compound'] = [sia.polarity_scores(x)['compound'] for x in tqdm(data['headline'])] # Se usa tqdm para mostrar una barra de progreso
# Clasifica el sentimiento numérico (compound) en positive, neutral o negative con la función definida al inicio.
data['VADER_sentiment'] = data['compound'].apply(label_sentiment);

data.head()

In [ ]:
# Guardamos los resultados
data.to_excel("LSNH_balanceado_VADER.xlsx", index=False)

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Asegurarse de que las etiquetas estén en el mismo formato (ej. capitalización)
data["label"] = data["label"].str.lower().str.strip()
data["VADER_sentiment"] = data["VADER_sentiment"].str.lower().str.strip()

# Calcular matriz de confusión
cm = confusion_matrix(data["label"], data["VADER_sentiment"], labels=["positive", "negative", "neutral"])

# Mostrar como DataFrame
cm_data = pd.DataFrame(cm, index=["True_Positive", "True_Negative", "True_Neutral"],
                        columns=["Pred_Positive", "Pred_Negative", "Pred_Neutral"])

print("Matriz de confusión:")
print(cm_data)

# Reporte de métricas
print("\nReporte de clasificación:")
print(classification_report(data["label"], data["VADER_sentiment"], digits=3))

# Visualización con heatmap
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Pred_Positive", "Pred_Negative", "Pred_Neutral"],
            yticklabels=["True_Positive", "True_Negative", "True_Neutral"])
plt.xlabel("PREDICCIONES")
plt.ylabel("VALORES REALES")
plt.title("Matriz de Confusión (Dataset Noticias - Balanceado)")
plt.show()

## Dataset sobre tweets de carácter financiero
**Financial Tweets Sentiment**

URL: https://huggingface.co/datasets/TimKoornstra/financial-tweets-sentiment

In [ ]:
import pandas as pd

# Cargar el dataset
df_financial_tweets_sentiment = pd.read_parquet("train-00000-of-00001.parquet")

df_financial_tweets_sentiment.head()

In [ ]:
df_financial_tweets_sentiment.shape

In [ ]:
df_financial_tweets_sentiment.info()

In [ ]:
# Realizo un mapeo de etiquetas para visualizar mejor los datos
def map_labels(label):
    if label == 1:
        return "Bullish"   # Bullish
    elif label == 2:
        return "Bearish"   # Bearish
    else:
        return "Neutral"    # Neutral

df_financial_tweets_sentiment["sentiment_mapped"] = df_financial_tweets_sentiment["sentiment"].apply(map_labels)

df_financial_tweets_sentiment.head()

In [ ]:
import matplotlib.pyplot as plt

# Datos
counts = df_financial_tweets_sentiment['sentiment_mapped'].value_counts()
labels = counts.index

# Paleta de colores agradable (pastel o profesional)
colors = plt.cm.Pastel1(range(len(labels)))

# Gráfico
fig, ax = plt.subplots(figsize=(6, 6))
wedges, texts, autotexts = ax.pie(
    counts,
    labels=labels,
    autopct='%1.1f%%',
    startangle=90,
    colors=colors,
    textprops={'fontsize': 12, 'color': 'black'}
)

# Mejorar visibilidad de porcentajes
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_fontweight('bold')

# Título con formato profesional
ax.set_title(
    "Distribución de sentimiento en dataset original",
    fontsize=12,
    fontweight='bold',
    pad=20
)

# Mantener el pastel como círculo perfecto
ax.axis('equal')

# Quitar el borde de las etiquetas para un look limpio
plt.setp(texts, fontweight='medium')

plt.show()

In [ ]:
# BALANCEO EL DATASET
# Comprobamos distribución inicial
print("Distribución original:")
print(df_financial_tweets_sentiment['sentiment_mapped'].value_counts(normalize=True) * 100)

# Determinamos tamaño de la clase minoritaria (Bearish)
min_class = "Bearish"
min_count = df_financial_tweets_sentiment[df_financial_tweets_sentiment['sentiment_mapped'] == min_class].shape[0]
print(f"\nTamaño de la clase minoritaria ({min_class}): {min_count}")

# Tomamos todos los textos de la clase minoritaria
df_min = df_financial_tweets_sentiment[df_financial_tweets_sentiment['sentiment_mapped'] == min_class]

# Submuestreamos aleatoriamente las otras clases para igualar tamaño
balanced_parts = [df_min]  # empezamos con la clase minoritaria

for label in ["Bullish", "Neutral"]:
    df_label = df_financial_tweets_sentiment[df_financial_tweets_sentiment['sentiment_mapped'] == label].sample(
        n=min_count,
        random_state=42
    )
    balanced_parts.append(df_label)

# Combinamos en un dataset balanceado
df_financial_tweets_sentiment_balanceado = pd.concat(balanced_parts).sample(frac=1, random_state=42).reset_index(drop=True)

# Comprobamos distribución final
print("\nDistribución balanceada:")
print(df_financial_tweets_sentiment_balanceado['sentiment_mapped'].value_counts())

# Guardamos el dataset balanceado
df_financial_tweets_sentiment_balanceado.to_excel("Dataset_financial_tweets_sentiment_balanced.xlsx", index=False)

print("\nDataset balanceado guardado correctamente.")

In [ ]:
# Realizo un mapeo de etiquetas antes de comparar, para así usar exactamente el mismo formato que generan mis modelos
def map_labels(label):
    if label == 1:
        return "POSITIVE"   # Bullish
    elif label == 2:
        return "NEGATIVE"   # Bearish
    else:
        return "NEUTRAL"    # Neutral

df_financial_tweets_sentiment_balanceado["sentiment_mapped"] = df_financial_tweets_sentiment_balanceado["sentiment"].apply(map_labels)

df_financial_tweets_sentiment_balanceado.head()

### 1. Modelo FinBERT

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

# Cargar FinBERT desde Hugging Face
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Crear el pipeline de FinBERT
finbert = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=False, truncation=True)

# Aplicar FinBERT a cada titular
df_financial_tweets_sentiment_balanceado['FinBERT_sentiment'] = df_financial_tweets_sentiment_balanceado['tweet'].apply(lambda x: finbert(x)[0]['label'])
df_financial_tweets_sentiment_balanceado['confidence'] = df_financial_tweets_sentiment_balanceado['tweet'].apply(lambda x: finbert(x)[0]['score'])

# Mostrar resultados
print(df_financial_tweets_sentiment_balanceado)
df_financial_tweets_sentiment_balanceado.to_excel("FTS_balanceado_FinBERT.xlsx", index=False)

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Asegurarse de que las etiquetas estén en el mismo formato (ej. capitalización)
df_financial_tweets_sentiment_balanceado["sentiment_mapped"] = df_financial_tweets_sentiment_balanceado["sentiment_mapped"].str.lower().str.strip()
df_financial_tweets_sentiment_balanceado["FinBERT_sentiment"] = df_financial_tweets_sentiment_balanceado["FinBERT_sentiment"].str.lower().str.strip()

# Calcular matriz de confusión
cm = confusion_matrix(df_financial_tweets_sentiment_balanceado["sentiment_mapped"], df_financial_tweets_sentiment_balanceado["FinBERT_sentiment"], labels=["positive", "negative", "neutral"])

# Mostrar como DataFrame
cm_df_financial_tweets_sentiment_balanceado = pd.DataFrame(cm, index=["True_Positive", "True_Negative", "True_Neutral"],
                        columns=["Pred_Positive", "Pred_Negative", "Pred_Neutral"])

print("Matriz de confusión:")
print(cm_df_financial_tweets_sentiment_balanceado)

# Reporte de métricas
print("\nReporte de clasificación:")
print(classification_report(df_financial_tweets_sentiment_balanceado["sentiment_mapped"], df_financial_tweets_sentiment_balanceado["FinBERT_sentiment"], digits=3))

# Visualización con heatmap
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Pred_Positive", "Pred_Negative", "Pred_Neutral"],
            yticklabels=["True_Positive", "True_Negative", "True_Neutral"])
plt.xlabel("PREDICCIONES")
plt.ylabel("VALORES REALES")
plt.title("Matriz de Confusión (Dataset Tweets - Balanceado)")
plt.show()

### 2. Modelo DistilRoBERTa + Fine-tuning

In [ ]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Cargar el modelo desde Hugging Face
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Creamos el pipeline con el modelo
sentiment_model = pipeline(
    "sentiment-analysis",
    model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis",
    truncation=True  # distilroberta solo admite secuencias de máximo 512 tokens, pero en tus datos (tweet) hay textos más largos (587 tokens en algún tweet). Eso dispara el RuntimeError en la capa de embeddings y da error en la ejecución. SOLUCIÓN: Truncar cualquier texto más largo al máximo permitido.
)

# Función para analizar sentimiento
def analyze_sentiment(text):
    result = sentiment_model(text)[0]
    return pd.Series([result['label'], result['score']])

# Aplicar análisis a los titulares
df_financial_tweets_sentiment_balanceado[['DistilRoBERTa_sentiment', 'confidence']] = df_financial_tweets_sentiment_balanceado['tweet'].apply(analyze_sentiment)

print(df_financial_tweets_sentiment_balanceado)
df_financial_tweets_sentiment_balanceado.to_excel("FTS_balanceado_DistilRoBERTa.xlsx", index=False)

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Asegurarse de que las etiquetas estén en el mismo formato (ej. capitalización)
df_financial_tweets_sentiment_balanceado["sentiment_mapped"] = df_financial_tweets_sentiment_balanceado["sentiment_mapped"].str.lower().str.strip()
df_financial_tweets_sentiment_balanceado["DistilRoBERTa_sentiment"] = df_financial_tweets_sentiment_balanceado["DistilRoBERTa_sentiment"].str.lower().str.strip()

# Calcular matriz de confusión
cm = confusion_matrix(df_financial_tweets_sentiment_balanceado["sentiment_mapped"], df_financial_tweets_sentiment_balanceado["DistilRoBERTa_sentiment"], labels=["positive", "negative", "neutral"])

# Mostrar como DataFrame
cm_df_financial_tweets_sentiment_balanceado = pd.DataFrame(cm, index=["True_Positive", "True_Negative", "True_Neutral"],
                        columns=["Pred_Positive", "Pred_Negative", "Pred_Neutral"])

print("Matriz de confusión:")
print(cm_df_financial_tweets_sentiment_balanceado)

# Reporte de métricas
print("\nReporte de clasificación:")
print(classification_report(df_financial_tweets_sentiment_balanceado["sentiment_mapped"], df_financial_tweets_sentiment_balanceado["DistilRoBERTa_sentiment"], digits=3))

# Visualización con heatmap
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Pred_Positive", "Pred_Negative", "Pred_Neutral"],
            yticklabels=["True_Positive", "True_Negative", "True_Neutral"])
plt.xlabel("PREDICCIONES")
plt.ylabel("VALORES REALES")
plt.title("Matriz de Confusión (Dataset Tweets - Balanceado)")
plt.show()

### 3. Modelo BERTweet

In [ ]:
# Importar el modelo y tokenizer de BERTweet
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
from tqdm import tqdm

# Modelo fine-tuned para análisis de sentimiento
MODEL_NAME = "finiteautomata/bertweet-base-sentiment-analysis" # Este modelo ya está afinado específicamente para clasificación de sentimiento en tweets
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# Creamos una función para predecir sentimiento
def predict_sentiment(tweet: str) -> str:
    inputs = tokenizer(tweet, return_tensors="pt", truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = F.softmax(logits, dim=1)
        label_id = torch.argmax(probs, dim=1).item()

    labels = model.config.id2label
    return labels[label_id]

# Lo aplico a mis tweets en formato original (sin preprocesado) porque el modelo de Hugging Face está entrenado para interpretar todo (emojis, menciones, etc.) como señales de sentimiento
tqdm.pandas() # Para mostrar una barra de progreso
# Convierto los valores a cadenas y reemplazo los NaN por texto vacío
df_financial_tweets_sentiment_balanceado['tweet'] = df_financial_tweets_sentiment_balanceado['tweet'].fillna('').astype(str)
# Creamos una nueva columna llamada bertweet_sentiment con etiquetas como NEG, NEU, POS llamando a la función anterior
df_financial_tweets_sentiment_balanceado['BERTweet_sentiment'] = df_financial_tweets_sentiment_balanceado['tweet'].progress_apply(predict_sentiment)

# Guardamos los resultados
df_financial_tweets_sentiment_balanceado.to_excel("FTS_balanceado_BERTweet.xlsx", index=False)
df_financial_tweets_sentiment_balanceado.head()

In [ ]:
# Tengo que mapear la etiqueta de sentiminto generada por mi modelo
def map_labels(label):
    if label == "POS":
        return "POSITIVE"
    elif label == "NEG":
        return "NEGATIVE"
    else:
        return "NEUTRAL"

df_financial_tweets_sentiment_balanceado["BERTweet_sentiment_mapped"] = df_financial_tweets_sentiment_balanceado["BERTweet_sentiment"].apply(map_labels)

df_financial_tweets_sentiment_balanceado.head()

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Asegurarse de que las etiquetas estén en el mismo formato (ej. capitalización)
df_financial_tweets_sentiment_balanceado["sentiment_mapped"] = df_financial_tweets_sentiment_balanceado["sentiment_mapped"].str.lower().str.strip()
df_financial_tweets_sentiment_balanceado["BERTweet_sentiment_mapped"] = df_financial_tweets_sentiment_balanceado["BERTweet_sentiment_mapped"].str.lower().str.strip()

# Calcular matriz de confusión
cm = confusion_matrix(df_financial_tweets_sentiment_balanceado["sentiment_mapped"], df_financial_tweets_sentiment_balanceado["BERTweet_sentiment_mapped"], labels=["positive", "negative", "neutral"])

# Mostrar como DataFrame
cm_df_financial_tweets_sentiment_balanceado = pd.DataFrame(cm, index=["True_Positive", "True_Negative", "True_Neutral"],
                        columns=["Pred_Positive", "Pred_Negative", "Pred_Neutral"])

print("Matriz de confusión:")
print(cm_df_financial_tweets_sentiment_balanceado)

# Reporte de métricas
print("\nReporte de clasificación:")
print(classification_report(df_financial_tweets_sentiment_balanceado["sentiment_mapped"], df_financial_tweets_sentiment_balanceado["BERTweet_sentiment_mapped"], digits=3))

# Visualización con heatmap
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Pred_Positive", "Pred_Negative", "Pred_Neutral"],
            yticklabels=["True_Positive", "True_Negative", "True_Neutral"])
plt.xlabel("PREDICCIONES")
plt.ylabel("VALORES REALES")
plt.title("Matriz de Confusión (Dataset Tweets - Balanceado)")
plt.show()

### 4. Modelo VADER

In [ ]:
# Instalamos las librerías necesarias
!pip install pyspellchecker
!pip install scattertext
!pip install nltk
!pip install -U kaleido

In [ ]:
# Import Data Preprocessing and Wrangling libraries
import re
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from datetime import datetime
import dateutil.parser

# Import NLP Libraries
import nltk
from spellchecker import SpellChecker
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

# Import Visualization Libraries
import plotly.offline as pyo
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import seaborn as sns
import scattertext as st
from IPython.display import IFrame
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import random

# Downloading periphrals
nltk.download('vader_lexicon')
nltk.download('stopwords')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Inicializamos las herramientas

# Para visualizaciones con seaborn
sns.set_style('darkgrid')

# Análisis de sentimiento con VADER
sia = SIA()

# Corrector ortográfico (puede ayudar antes del análisis de sentimiento)
spell = SpellChecker()

# Para mostrar gráficos Plotly en notebooks
pyo.init_notebook_mode()

In [ ]:
""" Aplicamos pipeline de preprocesamiento para modelo VADER (limpieza más profunda)"""

# Se crea una copia del DataFrame original df_tweets para no modificarlo directamente
data = df_financial_tweets_sentiment_balanceado.copy()
# Se añade una columna original_tweet con el texto sin procesar a modo de backup para comparar
data['original_tweet'] = df_financial_tweets_sentiment_balanceado['tweet']
# Reemplaza los valores nulos (NaN) en la columna "text" por cadenas vacías '' y convierte todo el contenido de la columna "text" a tipo string.
# Esto sirve para evitar que el modelo falle al encontrarse con un NaN y para garantizar que todo se maneje como string.
data['tweet'] = data['tweet'].fillna('').astype(str)


# Aplica la función de limpieza a la columna "text"
data['tweet'] = data['tweet'].apply(limpieza_vader)
# Eliminamos duplicados
data.drop_duplicates(subset=["tweet"], inplace=True)
# Reseteamos el índice por si se ha eliminado alguna fila
data.reset_index(drop=True, inplace=True)

data.head()

In [ ]:
import textwrap

# Ancho máximo de cada línea (ejemplo: 80 caracteres)
width = 80

print("="*80)
print("tweet ANTES de preprocesamiento:")
print("="*80)
print(textwrap.fill(data.original_tweet[0], width=width))  # Aquí hace el salto de línea
print("\n" + "="*80)
#print(data.original_tweet[0])
print("tweet DESPUÉS de preprocesamiento:")
print(80*"=")
print(textwrap.fill(data.tweet[0], width=width))  # También aquí
#print(data.text[0])

In [ ]:
# Esta función convierte un valor de sentimiento (entre -1 y 1, generado por VADER) en una etiqueta categórica
def label_sentiment(x:float):
    if x < -0.05 : return 'negative'  # negative si es muy bajo
    if x > 0.05 : return 'positive'   # positive si es alto
    return 'neutral'                  # neutral si está entre ambos umbrales

# EXTRACCIÓN DE CARACTERÍSTICAS del texto
# Extrae todas las palabras de cada tweet usando regex, eliminando puntuación.
data['words'] = data.tweet.apply(lambda x:re.findall(r'\w+', x ))
# Usa SpellChecker para detectar palabras mal escritas
data['errors'] = data.words.apply(spell.unknown)
# Cuenta cuántos errores ortográficos y cuántas palabras hay por tweet
data['errors_count'] = data.errors.apply(len)
data['words_count'] = data.words.apply(len)
# Longitud del tweet (en caracteres)
data['sentence_length'] = data.tweet.apply(len)

# Análisis de sentimiento para cada tweet
# Aplica VADER (SentimentIntensityAnalyzer = SIA) a cada tweet para obtener el sentimiento compuesto (compound score), que va de -1 (negativo) a 1 (positivo).
data['compound'] = [sia.polarity_scores(x)['compound'] for x in tqdm(data['tweet'])] # Se usa tqdm para mostrar una barra de progreso
# Clasifica el sentimiento numérico (compound) en positive, neutral o negative con la función definida al inicio.
data['VADER_sentiment'] = data['compound'].apply(label_sentiment);

data.head()

In [ ]:
# Guardamos los resultados
data.to_excel("FTS_balanceado_VADER.xlsx", index=False)

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Asegurarse de que las etiquetas estén en el mismo formato (ej. capitalización)
data["sentiment_mapped"] = data["sentiment_mapped"].str.lower().str.strip()
data["VADER_sentiment"] = data["VADER_sentiment"].str.lower().str.strip()

# Calcular matriz de confusión
cm = confusion_matrix(data["sentiment_mapped"], data["VADER_sentiment"], labels=["positive", "negative", "neutral"])

# Mostrar como DataFrame
cm_data = pd.DataFrame(cm, index=["True_Positive", "True_Negative", "True_Neutral"],
                        columns=["Pred_Positive", "Pred_Negative", "Pred_Neutral"])

print("Matriz de confusión:")
print(cm_data)

# Reporte de métricas
print("\nReporte de clasificación:")
print(classification_report(data["sentiment_mapped"], data["VADER_sentiment"], digits=3))

# Visualización con heatmap
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Pred_Positive", "Pred_Negative", "Pred_Neutral"],
            yticklabels=["True_Positive", "True_Negative", "True_Neutral"])
plt.xlabel("PREDICCIONES")
plt.ylabel("VALORES REALES")
plt.title("Matriz de Confusión (Dataset Tweets - Balanceado)")
plt.show()

## Visualización comparativa de modelos

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Métricas
metrics = ["Accuracy", "F1 Negative", "F1 Neutral", "F1 Pos", "F1 Macro", "Eficiencia Computacional"]

# ---------- Datos Titulares Original ----------
FinBERT_tit =       [0.557, 0.643, 0.577, 0.487, 0.569, 8]    # 8 min
DistilRoBERTa_tit = [0.545, 0.626, 0.563, 0.483, 0.558, 2]    # 2 min
BERTweet_tit =      [0.503, 0.626, 0.519, 0.427, 0.524, 42]   # 42 min
VADER_tit =         [0.430, 0.404, 0.396, 0.471, 0.424, 0.05] # 3 seg ≈ 0.05 min

# ---------- Datos Tweets Original ----------
FinBERT_tw =       [0.464, 0.498, 0.507, 0.374, 0.460, 11]   # 11 min
DistilRoBERTa_tw = [0.502, 0.519, 0.527, 0.456, 0.501, 4]    # 4 min
BERTweet_tw =      [0.512, 0.542, 0.522, 0.483, 0.516, 73]   # 73 min
VADER_tw =         [0.451, 0.436, 0.420, 0.487, 0.448, 0.12] # 7 seg ≈ 0.12 min

# Normalizar e invertir coste → eficiencia
# Extraer todos los tiempos
time_values = [
    FinBERT_tit[-1], DistilRoBERTa_tit[-1], BERTweet_tit[-1], VADER_tit[-1],
    FinBERT_tw[-1], DistilRoBERTa_tw[-1], BERTweet_tw[-1], VADER_tw[-1]
]

# Normalización logarítmica + min-max
log_times = [np.log(t + 1) for t in time_values]
min_log, max_log = min(log_times), max(log_times)

for model in [FinBERT_tit, DistilRoBERTa_tit, BERTweet_tit, VADER_tit,
              FinBERT_tw, DistilRoBERTa_tw, BERTweet_tw, VADER_tw]:
    model[-1] = ((max_log - np.log(model[-1] + 1)) / (max_log - min_log)) + 0.1


# Ángulos radar
angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
angles += angles[:1]

def close_values(values):
    return values + values[:1]

# Función para dibujar cada radar
def plot_radar(ax, title, models):
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(metrics)
    ax.set_title(title, size=18, y=1.08)

    for name, values in models.items():
        vals = close_values(values)
        ax.plot(angles, vals, label=name)
        ax.fill(angles, vals, alpha=0.1)

# Crear la figura con dos subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7), subplot_kw=dict(polar=True))

# Radar Titulares Original
plot_radar(ax1, "News_O", {
    "FinBERT": FinBERT_tit,
    "DistilRoBERTa": DistilRoBERTa_tit,
    "BERTweet": BERTweet_tit,
    "VADER": VADER_tit
})

# Radar Tweets Original
plot_radar(ax2, "Tweet_O", {
    "FinBERT": FinBERT_tw,
    "DistilRoBERTa": DistilRoBERTa_tw,
    "BERTweet": BERTweet_tw,
    "VADER": VADER_tw
})

# Eliminar duplicados en la leyenda
handles, labels = fig.axes[0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc="upper center", bbox_to_anchor=(0.5, 1.1), ncol=1)

plt.show()

# Recolección de datos desde diferentes fuentes

## 1. PyGoogleNews para obtención de titulares de noticias

In [ ]:
!pip install feedparser
!pip install dateparser
!pip install plotly
import pygooglenews   # Importo mi módulo pygooglenews.py

In [ ]:
import pandas as pd
from pygooglenews import GoogleNews  # Importo la clase GoogleNews de mi módulo pygooglenews.py
import datetime  # Para manejo de fechas
import time  # Para pausar entre solicitudes

# Crea un objeto buscador de noticias para consultar noticias desde Google News
gn = GoogleNews()

# Esta función busca noticias relacionadas con un término (por ejemplo: "Tesla OR TSLA")
# entre una determinada fecha, dividiendo la búsqueda en bloques de 30 días para evitar límites
# en la cantidad de resultados que Google News devuelve de una sola vez.
def get_news(search):
  stories = []  # Lista

  start_date = datetime.date(2025,8,27)
  end_date = datetime.date(2025,8,29)
  delta = datetime.timedelta(days=1) # Con esta línea recorro el año día a día para mejorar la precisión diaria, realizando 365 consultas a Google News (OJO con los bloqueos por el exceso de peticiones).
  date = start_date

  while date <= end_date:
        try:
            print(f"Buscando noticias del {date}...")
            result = gn.search(
                search,
                from_=date.strftime('%Y-%m-%d'),
                to_=(date + delta).strftime('%Y-%m-%d')
            )

            newsitems = result['entries']

            for item in newsitems:
                story = {
                    'title': item.title,
                    'published': datetime.datetime.strptime(item.published, '%a, %d %b %Y %X GMT')
                }
                stories.append(story)

        except Exception as e:
            print(f"Error en {date}: {e}")

        # Esperar 2 segundos antes de la siguiente petición
        time.sleep(2)
        date += delta

  return stories

# Convertimos la lista de diccionarios stories en un DataFrame,
# a la par que llamamos a la función get_news con el “[Nombre Compañia] OR [Ticker]” como parámetro.
# Ticker = etiqueta de cotización con el que opera en bolsa.
df = pd.DataFrame(get_news('Amazon OR AMZN'))

# Agregar columna con solo la fecha
df['date'] = df['published'].dt.date

# Mostrar cantidad de noticias por día
news_per_day = df.groupby('date').size()
print(news_per_day)

In [ ]:
# Se guardan los resultados de las noticias recolectadas
df.to_excel("noticias_PyGoogleNews.xlsx", index=False)

In [ ]:
# Estas líneas es por si queremos filtrar un día concreto de noticias
df['date'] = df['published'].dt.date
df = df[df['date'] == datetime.date(2025, 8, 28)]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Preparar los datos
df['date'] = df['published'].dt.date
news_per_day = df.groupby('date').size().reset_index()
news_per_day.columns = ['date', 'count']
news_per_day['rolling_7'] = news_per_day['count'].rolling(window=7).mean()

# Crear figura
fig = go.Figure()

# Línea principal (noticias por día)
fig.add_trace(go.Scatter(
    x=news_per_day['date'],
    y=news_per_day['count'],
    mode='lines',
    name='Noticias por día',
    line=dict(color='royalblue')
))

# Media móvil de 7 días
fig.add_trace(go.Scatter(
    x=news_per_day['date'],
    y=news_per_day['rolling_7'],
    mode='lines',
    name='Media móvil (7 días)',
    line=dict(color='orange', dash='dash')
))

# Picos de noticias (top 5 días con más noticias)
top_days = news_per_day.nlargest(5, 'count')
fig.add_trace(go.Scatter(
    x=top_days['date'],
    y=top_days['count'],
    mode='markers+text',
    name='Picos de noticias',
    marker=dict(color='red', size=10),
    text=top_days['date'].astype(str),
    textposition='top center'
))

# =====================
# 🔹 Anotaciones de eventos relevantes (ejemplo: resultados financieros)
# =====================
eventos = {
    "2025-02-26": "Resultados Q4 2024",
    "2025-05-28": "Resultados Q1 2025",
    #"2025-07-23": "Resultados Q2 2025",
    #"2024-10-23": "Resultados Q3 2024",
    "2025-01-27": "Caída del 17% por noticia DeepSeek",
    "2025-03-18": "Conferencia anual GPU Technology Conference",
    #"2025-06-06": "Enfrentamiento público entre Elon Musk y Trump",
    #"2025-07-07": "Anuncio de Elon Musk sobre la creación del nuevo partido político llamado America Party"
}
for fecha, texto in eventos.items():
    fig.add_annotation(
        x=fecha,
        y=news_per_day.loc[news_per_day['date'] == pd.to_datetime(fecha).date(), 'count'].values[0],
        text=texto,
        showarrow=True,
        arrowhead=2,
        ax=40, ay=-40,
        bgcolor="white"
    )

"""# =====================
# 🔹 Sombreado de un periodo de interés (ejemplo: semana de alta cobertura)
# =====================
fig.add_vrect(
    x0="2024-05-10", x1="2024-05-20",
    fillcolor="lightgrey", opacity=0.3,
    layer="below", line_width=0,
    annotation_text="Alta cobertura",
    annotation_position="top left"
)"""

# Estética del gráfico
fig.update_layout(
    title='Cobertura diaria de noticias sobre NVIDIA (2025)',
    xaxis_title='Fecha',
    yaxis_title='Cantidad de noticias',
    hovermode='x unified',
    template='plotly_white',
    width=1000,
    height=500,
    legend=dict(
        orientation="h",
        yanchor="bottom", y=-0.3,
        xanchor="center", x=0.5
    )
)

fig.show()

## 2. Financial Modeling Prep (FMP) para obtención de titulares de últimas noticias

FMP es la abreviatura de Financial Modeling Prep, una plataforma que ofrece una API financiera robusta y flexible para acceder a datos bursátiles y económicos en tiempo real e históricos. Es ampliamente utilizada por desarrolladores, analistas financieros, investigadores y estudiantes para integrar información financiera en aplicaciones, hojas de cálculo y modelos de análisis.

Su API en versión gratuita solo permite:


*   Últimas 200 noticias financieras de las empresas que sean.
*   NO permite filtrar por empresa o por fecha.
*   Variedad de endpoints para poder hacer distintas peticiones (no solo noticias), como datos técnicos financieros, datos de empresa específica, etc. --> Ver todo los tipos en "https://site.financialmodelingprep.com/developer/docs/stable"

In [ ]:
import requests
import pandas as pd

API_KEY = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  # Reemplaza con tu API key real
tickers = 'TSLA'   # Al buscar noticias en la versión free no me permite filtrar por empresa
limit = 500  # El límite máximo de últimas noticias que me devuelve la API versión gratuita es 200

# Existen mucha variedad de endpoints: datos financieros, datos de empresa específica, noticias...
# La URL siguiente hace referencia al único endpoint de noticias que me permite la versión gratis (200 noticias máximo)
url = f'https://financialmodelingprep.com/stable/fmp-articles?page=0&limit={limit}&apikey={API_KEY}'


response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    # Convertir a DataFrame
    df_news = pd.DataFrame(data)
    print(df_news)
else:
    print("Error:", response.status_code, response.text)

In [ ]:
# Guardamos las noticias recientes extraídas
df_news.to_excel("ultimas_noticias_FMP.xlsx", index=False)

In [ ]:
df_news.head()

In [ ]:
# Filtro los 200 artículos por el ticker deseado
TICKER = 'TSLA'  # Ticker que quiera filtrar
FECHA_MINIMA = '2025-08-04'  # Fecha (incluida) a partir de la cual quiero obtener las noticias

# Convierto la columna 'date' a formato datetime
df_news['date'] = pd.to_datetime(df_news['date'])

# Filtro si la columna 'tickers' contiene ese símbolo. También filtro por fecha
df_FMP = df_news[
    (df_news['tickers'].apply(lambda x: TICKER in x if isinstance(x, list) else TICKER in str(x))) &
    (df_news['date'] >= pd.to_datetime(FECHA_MINIMA))
]

df_FMP.head()

## 3. YFinance para obtención de datos financieros

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf

df_precio = yf.download('AMZN', start = '2025-08-27', end = '2025-08-29', group_by='ticker')

# yfinance devuelve por defecto columnas anidadas (MultiIndex en columnas). Hay que realizar los siguientes pasos para eliminar el MultiIndex y obtener un dataframe normal
# 1. Resetear índice para tener 'Date' como columna
df_precio.reset_index(inplace=True)

# 2. Aplanar columnas (eliminar MultiIndex)
df_precio.columns = [col[1] if isinstance(col, tuple) else col for col in df_precio.columns]

# 3. Renombrar columna de fecha (si no se llama 'Date')
if 'Date' not in df_precio.columns:
    df_precio.rename(columns={df_precio.columns[0]: 'Date'}, inplace=True)

df_precio.head()

In [ ]:
df_precio.shape

In [ ]:
df_precio.info()

## 4. API Twitter/X para obtención de tweets y captar ruido social

### 4.1. Tweets relacionados con alguna compañía o activo financiero

In [ ]:
!pip install tweepy # Biblioteca de Python para interactuar con la API de Twitter

In [ ]:
import tweepy

# Pega aquí tu Bearer Token
BEARER_TOKEN = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

# Conexión con el cliente Tweepy
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Define la consulta: por ejemplo, tweets que mencionen "Tesla", en inglés, excluyendo retweets
query = 'Tesla lang:en -is:retweet'

# Búsqueda de hasta 10 tweets recientes
tweets = client.search_recent_tweets(query=query, max_results=10)

# Mostrar texto de tweets
if tweets.data:
    for tweet in tweets.data:
        print(tweet.text)
else:
    print("No se encontraron tweets")

### 4.2. Tweets recientes publicados por un usuario específico

In [ ]:
import tweepy

# Tus credenciales de la API de Twitter/X
BEARER_TOKEN = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

# Configurar cliente de tweepy con autenticación Bearer Token
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Nombre de usuario sin '@'
username = "elonmusk"

# Obtener user ID del usuario
user = client.get_user(username=username)
user_id = user.data.id

# Obtener tweets recientes del usuario (máximo 10)
tweets = client.get_users_tweets(id=user_id, max_results=10)

# Imprimir texto y fecha de los tweets
for tweet in tweets.data:
    print(tweet.created_at, "-", tweet.text)

## 5. Reddit para obtención de posts como fuente alternativa de sentimiento minorista

In [ ]:
!pip install praw  # Biblioteca de Python para interactuar con la API de Reddit

In [ ]:
import praw
import datetime

# Configura tu app en https://www.reddit.com/prefs/apps para obtener client_id, client_secret y user_agent
reddit = praw.Reddit(
    client_id='XXXXXXXXXXXXXXXXXXXXXXXX',
    client_secret='XXXXXXXXXXXXXXXXXXXXXXXX',
    user_agent='XXXXXXXXXXXXXXXXXXXXXXXX'  # Es un identificador para que Reddit sepa quién hace la petición, puede ser cualquier texto descriptivo.
)

# Elige un subreddit, por ejemplo r/WallStreetBets o r/investing
subreddit = reddit.subreddit('wallstreetbets')

# Lista para almacenar los datos
posts_data = []

# Obtener los 500 posts más recientes --> Devuelve como máximo los 1.000 aproximadamente más recientes
for post in subreddit.new(limit=500):
    posts_data.append({
        'title': post.title,
        'text': post.selftext,
        'created': datetime.datetime.fromtimestamp(post.created_utc),
        'url': post.url
    })

# Convertimos a DataFrame
df_reddit = pd.DataFrame(posts_data)

# Mostramos los primeros resultados
print(df_reddit.head())

In [ ]:
df_reddit.head()

In [ ]:
df_reddit.shape

# Resultados y Estudios

## 1.	Titulares de noticias por compañía y periodo específico vs Precio de acciones

En la obtención de datos seleccionamos previamente:

1.   Una compañía específica.
2.   Un rango de fechas específico.

In [ ]:
# Asignamos un valor numérico al sentimiento
sentiment_map = {'positive': 1, 'neutral': 0, 'negative': -1}
df['sentiment_score'] = df['sentiment'].map(sentiment_map)

# Agrupamos por fecha (media diaria del sentimiento)
df_sentiment_daily = df.groupby('date')['sentiment_score'].mean().reset_index()

# Convertimos la columna "published", que es de tipo objeto a tipo datetime
df_sentiment_daily['date'] = pd.to_datetime(df_sentiment_daily['date'])

df_sentiment_daily.head()

In [ ]:
# JOIN: Unimos sentimiento con precios por fecha
df_merged = pd.merge(df_precio, df_sentiment_daily, left_on='Date', right_on='date', how='left')

# Rellenar días sin noticias con 0 (neutral)
df_merged['sentiment_score'] = df_merged['sentiment_score'].fillna(0)

# Crear la columna de retorno diario
df_merged['daily_return'] = df_merged['Close'].pct_change()

df_merged.head()

In [ ]:
df_sentiment_daily.shape

In [ ]:
df_merged.shape

In [ ]:
# 1. Correlación numérica
correlation = df_merged[['sentiment_score', 'daily_return']].corr()
print(correlation)

In [ ]:
# 2. La correlación con desfase de 1 día es muy útil porque mide si el sentimiento de hoy tiene algún poder predictivo sobre el retorno de mañana.

# ───────────────────────────
# Crear columna con el retorno del día siguiente (desfase de 1 día)
# ───────────────────────────
df_merged['daily_return_lag1'] = df_merged['daily_return'].shift(-1)

# ───────────────────────────
# Calcular correlación de Pearson entre sentimiento actual y retorno del día siguiente
# ───────────────────────────
corr_lag1 = df_merged[['sentiment_score', 'daily_return_lag1']].corr().iloc[0,1]

print(f"Correlación (sentimiento_t vs. retorno_t+1): {corr_lag1:.4f}")

In [ ]:
# 3. Gráfico de dispersión (scatter plot)
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_merged, x='sentiment_score', y='daily_return')
plt.title('Correlación entre sentimiento de noticias y retorno diario de TSLA')
plt.xlabel('Sentimiento promedio (DistilRoBERTa)')
plt.ylabel('Retorno diario (%)')
plt.axhline(0, color='gray', linestyle='--')
plt.grid(True)
plt.show()

In [ ]:
# 4. Gráfico combinado en el tiempo
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

# Copia del dataframe para no modificar el original
df_plot = df_merged.copy()

# Media móvil de 7 días para suavizar el sentimiento
df_plot['sentiment_rolling'] = df_plot['sentiment_score'].rolling(7).mean()

# Crear figura y ejes
fig, ax1 = plt.subplots(figsize=(14, 7))

# --- Eje 1: Precio de cierre ---
ax1.plot(df_plot['Date'], df_plot['Close'], color='blue', label='Precio de cierre')
ax1.set_ylabel('Precio de acción (USD)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# --- Eje 2: Sentimiento ---
ax2 = ax1.twinx()

# Dibujar el sentimiento diario con colores (verde si >0, rojo si <0)
colors = np.where(df_plot['sentiment_score'] >= 0, 'green', 'red')
ax2.bar(df_plot['Date'], df_plot['sentiment_score'],
        color=colors, alpha=0.3, width=1.0)

# Añadir línea suavizada (media móvil 7 días)
ax2.plot(df_plot['Date'], df_plot['sentiment_rolling'],
         color='orange', linestyle='--', linewidth=2, label='Sentimiento (media móvil 7 días)')

ax2.set_ylabel('Sentimiento promedio (sentiment_score)', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# --- Leyenda manual para diferenciar positivo y negativo ---
pos_patch = mpatches.Patch(color='green', alpha=0.3, label='Sentimiento positivo')
neg_patch = mpatches.Patch(color='red', alpha=0.3, label='Sentimiento negativo')

# --- Título y estética ---
plt.title('Evolución del precio de NVDA y sentimiento de titulares de noticias (2025)', fontsize=14, fontweight='bold')
fig.tight_layout()

# Leyendas combinadas
lines_labels = [ax.get_legend_handles_labels() for ax in [ax1, ax2]]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
# Añadir parches personalizados
lines += [pos_patch, neg_patch]
labels += [pos_patch.get_label(), neg_patch.get_label()]

ax1.legend(lines, labels, loc='lower right')

plt.show()

In [ ]:
# 5. Agrupación semanal para mostrar cómo evoluciona el sentimiento semanal frente al precio semanal

# Agrupar semanalmente promediando valores numéricos
df_weekly = df_merged.set_index('Date').resample('W').mean(numeric_only=True).reset_index()

# Calcular correlación semanal
correlacion_semanal = df_weekly[['sentiment_score', 'daily_return']].corr().iloc[0, 1]
print(f"📅 Correlación semanal entre sentimiento y retorno: {correlacion_semanal:.4f}")

In [ ]:
# 6. Vamos a comprobar si el sentimiento de la semana anterior predice el retorno de la semana actual.
# Esto es muy útil para evaluar si el sentimiento anticipa movimientos de precio.
# Para ello aplicamos un desfase (lag).

# Crear una nueva columna con el sentimiento de la semana anterior
df_weekly['sentiment_lag1'] = df_weekly['sentiment_score'].shift(1)

# Calcular correlación entre sentimiento lag y retorno actual
correlacion_lag1 = df_weekly[['sentiment_lag1', 'daily_return']].corr().iloc[0, 1]
print(f"📊 Correlación con desfase 1 semana: {correlacion_lag1:.4f}")

In [ ]:
# 7. Análisis comparativo MISMO DÍA

import pandas as pd
import matplotlib.pyplot as plt

# Asegurarse de que el DataFrame esté ordenado por fecha
df_merged = df_merged.sort_values(by='Date')

# ────────────────
# Filtrar días por sentimiento y dirección del precio
# ────────────────
positivos = df_merged[df_merged['sentiment_score'] > 0]
negativos = df_merged[df_merged['sentiment_score'] < 0]

subidas_positivas = positivos[positivos['daily_return'] > 0]
bajadas_negativas = negativos[negativos['daily_return'] < 0]

# ────────────────
# Calcular porcentajes de coincidencia
# ────────────────
porcentaje_pos = len(subidas_positivas) / len(positivos) * 100 if len(positivos) > 0 else 0
porcentaje_neg = len(bajadas_negativas) / len(negativos) * 100 if len(negativos) > 0 else 0

# ────────────────
# Crear tabla de resultados
# ────────────────
tabla_resultados = pd.DataFrame({
    'Sentimiento': ['Positivo', 'Negativo'],
    'Días totales': [len(positivos), len(negativos)],
    'Mov. esperado': [len(subidas_positivas), len(bajadas_negativas)],
    '% coincidencia': [porcentaje_pos, porcentaje_neg]
})

print(tabla_resultados)

# ────────────────
# Gráfico de barras visual
# ────────────────
plt.figure(figsize=(8,5))
colores = ['green', 'red']
plt.bar(tabla_resultados['Sentimiento'], tabla_resultados['% coincidencia'], color=colores, alpha=0.7)
plt.ylabel('% días en que el precio se movió según el sentimiento')
plt.title('Relación entre sentimiento de noticias y movimiento del precio')
plt.ylim(0, 100)

# Añadir etiquetas de porcentaje encima de las barras
for i, val in enumerate(tabla_resultados['% coincidencia']):
    plt.text(i, val + 2, f"{val:.1f}%", ha='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# 8. Análisis comparativo DÍA SIGUIENTE

import pandas as pd
import matplotlib.pyplot as plt

# Asegurarse de que el DataFrame esté ordenado por fecha
df_merged = df_merged.sort_values(by='Date')

# ────────────────
# Desfase de 1 día: el sentimiento de hoy frente al retorno de mañana
# ────────────────
df_merged['daily_return_next'] = df_merged['daily_return'].shift(-1)

# Filtrar días por sentimiento y dirección del precio al día siguiente
positivos = df_merged[df_merged['sentiment_score'] > 0]
negativos = df_merged[df_merged['sentiment_score'] < 0]

subidas_positivas = positivos[positivos['daily_return_next'] > 0]
bajadas_negativas = negativos[negativos['daily_return_next'] < 0]

# ────────────────
# Calcular porcentajes de coincidencia
# ────────────────
porcentaje_pos = len(subidas_positivas) / len(positivos) * 100 if len(positivos) > 0 else 0
porcentaje_neg = len(bajadas_negativas) / len(negativos) * 100 if len(negativos) > 0 else 0

# ────────────────
# Crear tabla de resultados
# ────────────────
tabla_resultados = pd.DataFrame({
    'Sentimiento': ['Positivo', 'Negativo'],
    'Días totales': [len(positivos), len(negativos)],
    'Mov. esperado': [len(subidas_positivas), len(bajadas_negativas)],
    '% coincidencia': [porcentaje_pos, porcentaje_neg]
})

print(tabla_resultados)

# ────────────────
# Gráfico de barras visual
# ────────────────
plt.figure(figsize=(8,5))
colores = ['green', 'red']
plt.bar(tabla_resultados['Sentimiento'], tabla_resultados['% coincidencia'], color=colores, alpha=0.7)
plt.ylabel('% días en que el precio se movió según el sentimiento (día siguiente)')
plt.title('Relación entre sentimiento de noticias y movimiento del precio al día siguiente')
plt.ylim(0, 100)

# Añadir etiquetas de porcentaje encima de las barras
for i, val in enumerate(tabla_resultados['% coincidencia']):
    plt.text(i, val + 2, f"{val:.1f}%", ha='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# 9. Análisis comparativo SENTIMIENTOS EXTREMOS (> 0.2 ó < -0.2)

import pandas as pd
import matplotlib.pyplot as plt

# Asegurarse de que el DataFrame esté ordenado por fecha
df_merged = df_merged.sort_values(by='Date')

# ────────────────
# Desfase de 1 día
# ────────────────
df_merged['daily_return_next'] = df_merged['daily_return'].shift(-1)

# ────────────────
# Función para calcular coincidencia según umbrales de sentimiento extremo
# ────────────────
def porcentaje_coincidencia_extremo(df, sentiment_col, return_col, umbral_pos=0.2, umbral_neg=-0.2):
    positivos = df[df[sentiment_col] > umbral_pos]
    negativos = df[df[sentiment_col] < umbral_neg]

    subidas_positivas = positivos[positivos[return_col] > 0]
    bajadas_negativas = negativos[negativos[return_col] < 0]

    porcentaje_pos = len(subidas_positivas) / len(positivos) * 100 if len(positivos) > 0 else 0
    porcentaje_neg = len(bajadas_negativas) / len(negativos) * 100 if len(negativos) > 0 else 0

    tabla = pd.DataFrame({
        'Sentimiento': ['Muy positivo', 'Muy negativo'],
        'Días totales': [len(positivos), len(negativos)],
        'Mov. esperado': [len(subidas_positivas), len(bajadas_negativas)],
        '% coincidencia': [porcentaje_pos, porcentaje_neg]
    })
    return tabla

# ────────────────
# 1. Coincidencia mismo día
# ────────────────
tabla_mismo_dia_extremo = porcentaje_coincidencia_extremo(df_merged, 'sentiment_score', 'daily_return')
tabla_mismo_dia_extremo['Escenario'] = 'Mismo día'

# ────────────────
# 2. Coincidencia día siguiente
# ────────────────
tabla_dia_siguiente_extremo = porcentaje_coincidencia_extremo(df_merged, 'sentiment_score', 'daily_return_next')
tabla_dia_siguiente_extremo['Escenario'] = 'Día siguiente'

# ────────────────
# Combinar tablas
# ────────────────
tabla_completa_extremo = pd.concat([tabla_mismo_dia_extremo, tabla_dia_siguiente_extremo], ignore_index=True)
print(tabla_completa_extremo)

# ────────────────
# Gráfico comparativo
# ────────────────
plt.figure(figsize=(10,6))

anchura = 0.35
x = range(len(tabla_mismo_dia_extremo))

# Barras mismo día
plt.bar([i - anchura/2 for i in x],
        tabla_mismo_dia_extremo['% coincidencia'],
        width=anchura,
        color=['green','red'],
        alpha=0.7,
        label='Mismo día')

# Barras día siguiente
plt.bar([i + anchura/2 for i in x],
        tabla_dia_siguiente_extremo['% coincidencia'],
        width=anchura,
        color=['darkgreen','darkred'],
        alpha=0.7,
        label='Día siguiente')

plt.xticks(ticks=x, labels=tabla_mismo_dia_extremo['Sentimiento'])
plt.ylabel('% días en que el precio se movió según el sentimiento extremo')
plt.title('Relación entre sentimientos extremos y movimiento del precio de las acciones')
plt.ylim(0, 100)
plt.legend()

# Añadir etiquetas de porcentaje encima de las barras
for i, val in enumerate(tabla_mismo_dia_extremo['% coincidencia']):
    plt.text(i - anchura/2, val + 2, f"{val:.1f}%", ha='center', fontsize=10)
for i, val in enumerate(tabla_dia_siguiente_extremo['% coincidencia']):
    plt.text(i + anchura/2, val + 2, f"{val:.1f}%", ha='center', fontsize=10)

plt.tight_layout()
plt.show()

## 2.	Tweets con carácter financiero por compañía y periodo específico vs Precio de acciones

In [2]:
# Cargo el dataset con los tweets recolectados sobre Tesla y periodo del 01-01-2022 a 01-10-2022
import pandas as pd

df = pd.read_excel('Tweets_Recolectados_Tesla.xlsx')

In [ ]:
# Analizamos el sentimiento con el modelo BERTweet, pero podríamos haber usado cualquier otro
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
from tqdm import tqdm

# Modelo fine-tuned para análisis de sentimiento
MODEL_NAME = "finiteautomata/bertweet-base-sentiment-analysis" # Este modelo ya está afinado específicamente para clasificación de sentimiento en tweets
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# Creamos una función para predecir sentimiento
def predict_sentiment(tweet: str) -> str:
    inputs = tokenizer(tweet, return_tensors="pt", truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = F.softmax(logits, dim=1)
        label_id = torch.argmax(probs, dim=1).item()

    labels = model.config.id2label
    return labels[label_id]

# Lo aplico a mis tweets en formato original (sin preprocesado) porque el modelo de Hugging Face está entrenado para interpretar todo (emojis, menciones, etc.) como señales de sentimiento
tqdm.pandas() # Para mostrar una barra de progreso
# Convierto los valores a cadenas y reemplazo los NaN por texto vacío
df['Tweet'] = df['Tweet'].fillna('').astype(str)
# Creamos una nueva columna llamada bertweet_sentiment con etiquetas como NEG, NEU, POS llamando a la función anterior
df['bertweet_sentiment'] = df['Tweet'].progress_apply(predict_sentiment)

df.head()

In [ ]:
# Guardamos el resultado
df.to_excel("analisis_sentimiento_BERTweet.xlsx", index=False)

In [ ]:
# Asignamos un valor numérico al sentimiento
sentiment_map = {'POS': 1, 'NEU': 0, 'NEG': -1}
df['sentiment_score'] = df['bertweet_sentiment'].map(sentiment_map)

# Agrupamos por fecha (media diaria del sentimiento)
df_sentiment_daily = df.groupby('date')['sentiment_score'].mean().reset_index()

df_sentiment_daily.head()

In [ ]:
df_sentiment_daily.shape

In [ ]:
df_merged.shape

In [ ]:
# JOIN: Unimos sentimiento con precios por fecha
df_merged = pd.merge(df_precio, df_sentiment_daily, left_on='Date', right_on='date', how='left')

# Rellenar días sin tweets con 0 (neutral)
df_merged['sentiment_score'] = df_merged['sentiment_score'].fillna(0)

# Crear la columna de retorno diario
df_merged['daily_return'] = df_merged['Close'].pct_change()
df_merged.head()

In [ ]:
# 1. Correlación numérica
correlation = df_merged[['sentiment_score', 'daily_return']].corr()
print(correlation)

In [ ]:
# 2. La correlación con desfase de 1 día es muy útil porque mide si el sentimiento de hoy tiene algún poder predictivo sobre el retorno de mañana.

# ───────────────────────────
# Crear columna con el retorno del día siguiente (desfase de 1 día)
# ───────────────────────────
df_merged['daily_return_lag1'] = df_merged['daily_return'].shift(-1)

# ───────────────────────────
# Calcular correlación de Pearson entre sentimiento actual y retorno del día siguiente
# ───────────────────────────
corr_lag1 = df_merged[['sentiment_score', 'daily_return_lag1']].corr().iloc[0,1]

print(f"Correlación (sentimiento_t vs. retorno_t+1): {corr_lag1:.4f}")

In [ ]:
# 3. Diagrama de dispersión para ver la forma de la relación
import seaborn as sns
import matplotlib.pyplot as plt

sns.regplot(data=df_merged, x='sentiment_score', y='daily_return')
plt.title('Sentiment Score de tweets vs Daily Return (Tesla)')
plt.show()

In [ ]:
# 4. Gráfico combinado en el tiempo
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

# Copia del dataframe para no modificar el original
df_plot = df_merged.copy()

# Media móvil de 7 días para suavizar el sentimiento
df_plot['sentiment_rolling'] = df_plot['sentiment_score'].rolling(7).mean()

# Crear figura y ejes
fig, ax1 = plt.subplots(figsize=(14, 7))

# --- Eje 1: Precio de cierre ---
ax1.plot(df_plot['Date'], df_plot['Close'], color='blue', label='Precio de cierre')
ax1.set_ylabel('Precio de acción (USD)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# --- Eje 2: Sentimiento ---
ax2 = ax1.twinx()

# Dibujar el sentimiento diario con colores (verde si >0, rojo si <0)
colors = np.where(df_plot['sentiment_score'] >= 0, 'green', 'red')
ax2.bar(df_plot['Date'], df_plot['sentiment_score'],
        color=colors, alpha=0.3, width=1.0)

# Añadir línea suavizada (media móvil 7 días)
ax2.plot(df_plot['Date'], df_plot['sentiment_rolling'],
         color='orange', linestyle='--', linewidth=2, label='Sentimiento (media móvil 7 días)')

ax2.set_ylabel('Sentimiento promedio (sentiment_score)', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# --- Leyenda manual para diferenciar positivo y negativo ---
pos_patch = mpatches.Patch(color='green', alpha=0.3, label='Sentimiento positivo')
neg_patch = mpatches.Patch(color='red', alpha=0.3, label='Sentimiento negativo')

# --- Título y estética ---
plt.title('Evolución del precio de TSLA y sentimiento de tweets (2022)', fontsize=14, fontweight='bold')
fig.tight_layout()

# Leyendas combinadas
lines_labels = [ax.get_legend_handles_labels() for ax in [ax1, ax2]]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
# Añadir parches personalizados
lines += [pos_patch, neg_patch]
labels += [pos_patch.get_label(), neg_patch.get_label()]

ax1.legend(lines, labels, loc='lower right')

plt.show()

In [ ]:
# 5. Agrupación semanal para mostrar cómo evoluciona el sentimiento semanal frente al precio semanal

# Agrupar semanalmente promediando valores numéricos
df_weekly = df_merged.set_index('Date').resample('W').mean(numeric_only=True).reset_index()

# Calcular correlación semanal
correlacion_semanal = df_weekly[['sentiment_score', 'daily_return']].corr().iloc[0, 1]
print(f"📅 Correlación semanal entre sentimiento y retorno: {correlacion_semanal:.4f}")

In [ ]:
# 6. Vamos a comprobar si el sentimiento de la semana anterior predice el retorno de la semana actual.
# Esto es muy útil para evaluar si el sentimiento anticipa movimientos de precio.
# Para ello aplicamos un desfase (lag).

# Crear una nueva columna con el sentimiento de la semana anterior
df_weekly['sentiment_lag1'] = df_weekly['sentiment_score'].shift(1)

# Calcular correlación entre sentimiento lag y retorno actual
correlacion_lag1 = df_weekly[['sentiment_lag1', 'daily_return']].corr().iloc[0, 1]
print(f"📊 Correlación con desfase 1 semana: {correlacion_lag1:.4f}")

In [ ]:
# 7. Análisis comparativo MISMO DÍA y DÍA SIGUIENTE

import pandas as pd
import matplotlib.pyplot as plt

# Asegurarse de que el DataFrame esté ordenado por fecha
df_merged = df_merged.sort_values(by='Date')

# ────────────────
# Desfase de 1 día
# ────────────────
df_merged['daily_return_next'] = df_merged['daily_return'].shift(-1)

# ────────────────
# Función para calcular porcentaje de coincidencia
# ────────────────
def porcentaje_coincidencia(df, sentiment_col, return_col):
    positivos = df[df[sentiment_col] > 0]
    negativos = df[df[sentiment_col] < 0]

    subidas_positivas = positivos[positivos[return_col] > 0]
    bajadas_negativas = negativos[negativos[return_col] < 0]

    porcentaje_pos = len(subidas_positivas) / len(positivos) * 100 if len(positivos) > 0 else 0
    porcentaje_neg = len(bajadas_negativas) / len(negativos) * 100 if len(negativos) > 0 else 0

    tabla = pd.DataFrame({
        'Sentimiento': ['Positivo', 'Negativo'],
        'Días totales': [len(positivos), len(negativos)],
        'Mov. esperado': [len(subidas_positivas), len(bajadas_negativas)],
        '% coincidencia': [porcentaje_pos, porcentaje_neg]
    })
    return tabla

# ────────────────
# 1. Coincidencia mismo día
# ────────────────
tabla_mismo_dia = porcentaje_coincidencia(df_merged, 'sentiment_score', 'daily_return')
tabla_mismo_dia['Escenario'] = 'Mismo día'

# ────────────────
# 2. Coincidencia día siguiente
# ────────────────
tabla_dia_siguiente = porcentaje_coincidencia(df_merged, 'sentiment_score', 'daily_return_next')
tabla_dia_siguiente['Escenario'] = 'Día siguiente'

# ────────────────
# Combinar tablas
# ────────────────
tabla_completa = pd.concat([tabla_mismo_dia, tabla_dia_siguiente], ignore_index=True)
print(tabla_completa)

# ────────────────
# Gráfico comparativo
# ────────────────
plt.figure(figsize=(10,6))

anchura = 0.35
x = range(len(tabla_mismo_dia))

# Barras mismo día
plt.bar([i - anchura/2 for i in x],
        tabla_mismo_dia['% coincidencia'],
        width=anchura,
        color=['green','red'],
        alpha=0.7,
        label='Mismo día')

# Barras día siguiente
plt.bar([i + anchura/2 for i in x],
        tabla_dia_siguiente['% coincidencia'],
        width=anchura,
        color=['darkgreen','darkred'],
        alpha=0.7,
        label='Día siguiente')

plt.xticks(ticks=x, labels=tabla_mismo_dia['Sentimiento'])
plt.ylabel('% días en que el precio se movió según el sentimiento')
plt.title('Comparación de coincidencia entre sentimiento y movimiento del precio')
plt.ylim(0, 100)
plt.legend()

# Añadir etiquetas de porcentaje encima de las barras
for i, val in enumerate(tabla_mismo_dia['% coincidencia']):
    plt.text(i - anchura/2, val + 2, f"{val:.1f}%", ha='center', fontsize=10)
for i, val in enumerate(tabla_dia_siguiente['% coincidencia']):
    plt.text(i + anchura/2, val + 2, f"{val:.1f}%", ha='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# 8. Análisis comparativo SENTIMIENTO EXTREMO

import pandas as pd
import matplotlib.pyplot as plt

# Asegurarse de que el DataFrame esté ordenado por fecha
df_merged = df_merged.sort_values(by='Date')

# ────────────────
# Desfase de 1 día
# ────────────────
df_merged['daily_return_next'] = df_merged['daily_return'].shift(-1)

# ────────────────
# Función para calcular coincidencia según umbrales de sentimiento extremo
# ────────────────
def porcentaje_coincidencia_extremo(df, sentiment_col, return_col, umbral_pos=0.2, umbral_neg=-0.2):
    positivos = df[df[sentiment_col] > umbral_pos]
    negativos = df[df[sentiment_col] < umbral_neg]

    subidas_positivas = positivos[positivos[return_col] > 0]
    bajadas_negativas = negativos[negativos[return_col] < 0]

    porcentaje_pos = len(subidas_positivas) / len(positivos) * 100 if len(positivos) > 0 else 0
    porcentaje_neg = len(bajadas_negativas) / len(negativos) * 100 if len(negativos) > 0 else 0

    tabla = pd.DataFrame({
        'Sentimiento': ['Muy positivo', 'Muy negativo'],
        'Días totales': [len(positivos), len(negativos)],
        'Mov. esperado': [len(subidas_positivas), len(bajadas_negativas)],
        '% coincidencia': [porcentaje_pos, porcentaje_neg]
    })
    return tabla

# ────────────────
# 1. Coincidencia mismo día
# ────────────────
tabla_mismo_dia_extremo = porcentaje_coincidencia_extremo(df_merged, 'sentiment_score', 'daily_return')
tabla_mismo_dia_extremo['Escenario'] = 'Mismo día'

# ────────────────
# 2. Coincidencia día siguiente
# ────────────────
tabla_dia_siguiente_extremo = porcentaje_coincidencia_extremo(df_merged, 'sentiment_score', 'daily_return_next')
tabla_dia_siguiente_extremo['Escenario'] = 'Día siguiente'

# ────────────────
# Combinar tablas
# ────────────────
tabla_completa_extremo = pd.concat([tabla_mismo_dia_extremo, tabla_dia_siguiente_extremo], ignore_index=True)
print(tabla_completa_extremo)

# ────────────────
# Gráfico comparativo
# ────────────────
plt.figure(figsize=(10,6))

anchura = 0.35
x = range(len(tabla_mismo_dia_extremo))

# Barras mismo día
plt.bar([i - anchura/2 for i in x],
        tabla_mismo_dia_extremo['% coincidencia'],
        width=anchura,
        color=['green','red'],
        alpha=0.7,
        label='Mismo día')

# Barras día siguiente
plt.bar([i + anchura/2 for i in x],
        tabla_dia_siguiente_extremo['% coincidencia'],
        width=anchura,
        color=['darkgreen','darkred'],
        alpha=0.7,
        label='Día siguiente')

plt.xticks(ticks=x, labels=tabla_mismo_dia_extremo['Sentimiento'])
plt.ylabel('% días en que el precio se movió según el sentimiento extremo')
plt.title('Relación entre sentimientos extremos y movimiento del precio de las acciones')
plt.ylim(0, 100)
plt.legend()

# Añadir etiquetas de porcentaje encima de las barras
for i, val in enumerate(tabla_mismo_dia_extremo['% coincidencia']):
    plt.text(i - anchura/2, val + 2, f"{val:.1f}%", ha='center', fontsize=10)
for i, val in enumerate(tabla_dia_siguiente_extremo['% coincidencia']):
    plt.text(i + anchura/2, val + 2, f"{val:.1f}%", ha='center', fontsize=10)

plt.tight_layout()
plt.show()

## 3.	Tweets por usuario específico vs Precio de acciones

**Elon Musk Tweets 2010 to 2025**

URL: https://www.kaggle.com/datasets/dadalyndell/elon-musk-tweets-2010-to-2025-march?select=all_musk_posts.csv

In [ ]:
import pandas as pd

# Cargo el dataset con los tweets de Elon Musk entre el periodo de 2010 a 04/2025
df_tweets_ElonMusk = pd.read_csv('all_musk_posts.csv')

In [ ]:
df_tweets_ElonMusk.head()

In [ ]:
df_tweets_ElonMusk.shape

In [ ]:
df_tweets_ElonMusk.info()

In [ ]:
################################################################################
#                Búsqueda de nulos por columnas del Dataframe                  #
################################################################################

# Para poder encontrar el total de valores nulos haremos un tratamiento por columnas y, sumando dicho valor, obtendremos el total de nulos en cada columna
nulosPorColumna = df_tweets_ElonMusk.isnull().sum()

#Mostramos el resultado en una tabla
print("Número de valores nulos por columna:\n" + str(nulosPorColumna))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Visualizamos los valores nulos con Heatmap para una representación gráfica de los valores nulos
# Elegimos el color verde para representar que las columnas contienen un valor y en color rojo la representación de los valores nulos
sns.heatmap(df_tweets_ElonMusk.isnull(), cbar=False, cmap='RdYlGn_r')
plt.show()

In [ ]:
""" Para calcular el engagement_score, la mejor práctica es asumir que los valores nulos son ceros,
porque si no hay datos de interacción es probable que no haya habido interacciones o no se registraron. """
# Relleno con ceros
cols_to_fill = ['retweetCount', 'replyCount', 'likeCount', 'quoteCount', 'viewCount', 'bookmarkCount']
df_tweets_ElonMusk[cols_to_fill] = df_tweets_ElonMusk[cols_to_fill].fillna(0)

nulosPorColumna = df_tweets_ElonMusk.isnull().sum()
print("Número de valores nulos por columna:\n" + str(nulosPorColumna))

In [ ]:
# Calculamos el engagement de cada tweet

def calcula_engagement_score(row):
    # Filtro los tweets originales creados por Elon Musk (no me interesan los que él retuitea)
    if row['isRetweet']:
        return 0

    return round(
        2 * row['retweetCount'] +
        1.5 * row['replyCount'] +
        1.2 * row['likeCount'] +
        0.001 * row['viewCount'],
        2
    )

# Aplica la función a cada fila del DataFrame
df_tweets_ElonMusk['engagement_score'] = df_tweets_ElonMusk.apply(calcula_engagement_score, axis=1)

df_tweets_ElonMusk.head()

In [ ]:
# Filtro los tweets más influyentes
# Por ejemplo, ordeno por engagement_score de mayor a menor
df_tweets_ElonMusk_ordenado = df_tweets_ElonMusk.sort_values(by='engagement_score', ascending=False)

# Muestro los 10 tweets con más engagement
df_tweets_ElonMusk_ordenado[['createdAt', 'fullText', 'engagement_score']].head(10)

In [ ]:
# Filtramos por palabras clave por empresa/activo para buscar patrones con el movimiento del precio de acciones
keywords_tsla = ['tesla', 'tsla']
keywords_btc = ['bitcoin', 'btc']
keywords_doge = ['doge', 'dogecoin']

# Busca menciones relacionadas
tsla_tweets = df_tweets_ElonMusk_ordenado[df_tweets_ElonMusk_ordenado['fullText'].str.lower().str.contains('|'.join(keywords_tsla))]
btc_tweets = df_tweets_ElonMusk_ordenado[df_tweets_ElonMusk_ordenado['fullText'].str.lower().str.contains('|'.join(keywords_btc))]
doge_tweets = df_tweets_ElonMusk_ordenado[df_tweets_ElonMusk_ordenado['fullText'].str.lower().str.contains('|'.join(keywords_doge))]

In [ ]:
doge_tweets[['engagement_score', 'fullText', 'createdAt']].head()

In [ ]:
doge_tweets.shape

In [ ]:
# Convertimos la columna "createdAt", que es de tipo objeto a tipo datetime
doge_tweets['createdAt'] = pd.to_datetime(doge_tweets['createdAt'])

# Crear nueva columna 'date' con solo la fecha (sin hora)
doge_tweets['date'] = doge_tweets['createdAt'].dt.date

# Convertimos la columna "date", que es de tipo objeto a tipo datetime
doge_tweets['date'] = pd.to_datetime(doge_tweets['date'])

doge_tweets.info()

In [ ]:
# Agrupamos los engagement_score por fecha (por si hay varios tweets el mismo día) y se acumula el engagement_score diario
df_engagement_daily = doge_tweets.groupby('date')['engagement_score'].sum().reset_index()

df_engagement_daily.head(10)

In [ ]:
df_engagement_daily.shape

In [ ]:
df_merged.shape

In [ ]:
# JOIN: Unimos engagement con precios por fecha
df_merged = pd.merge(df_precio, df_engagement_daily, left_on='Date', right_on='date', how='left')

# Rellenar días sin tweets publicados por ElonMusk relacionados con DOGE con 0 (engagement nulo)
df_merged['engagement_score'] = df_merged['engagement_score'].fillna(0)

# Crear la columna de retorno diario
df_merged['daily_return'] = df_merged['Close'].pct_change()
df_merged.head()

In [ ]:
# 1. Correlación numérica
correlation = df_merged[['engagement_score', 'daily_return']].corr()
print(correlation)

In [ ]:
# 2. Gráfico de dispersión (scatter plot)
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_merged, x='engagement_score', y='daily_return')
plt.title('Correlación entre engagement_score de tweets de Elon Musk y retorno diario de DOGE')
plt.xlabel('engagement_score tweets')
plt.ylabel('Retorno diario (%)')
plt.axhline(0, color='gray', linestyle='--')
plt.grid(True)
plt.show()

In [ ]:
# 3. Gráfico combinado en el tiempo
import matplotlib.pyplot as plt
import numpy as np

# Copia del dataframe para no modificar el original
df_plot = df_merged.copy()

# Media móvil de 7 días para suavizar el engagement
df_plot['engagement_rolling'] = df_plot['engagement_score'].rolling(7).mean()

# Crear figura y ejes
fig, ax1 = plt.subplots(figsize=(14, 7))

# --- Eje 1: Precio de cierre ---
ax1.plot(df_plot['Date'], df_plot['Close'], color='#1f77b4', linewidth=2, label='Precio de cierre')
ax1.set_ylabel('Precio de acción (USD)', color='#1f77b4', fontsize=12)
ax1.tick_params(axis='y', labelcolor='#1f77b4')
ax1.grid(axis='x', linestyle='--', alpha=0.3)

# --- Eje 2: Engagement ---
ax2 = ax1.twinx()

# Línea de engagement diario más visible
ax2.plot(df_plot['Date'], df_plot['engagement_score'],
         color='red', linewidth=1.5, alpha=0.3, label='Engagement')

# Línea de media móvil discontinua
ax2.plot(df_plot['Date'], df_plot['engagement_rolling'],
         color='#ff7f0e', linestyle='--', linewidth=2.5, label='Engagement (media móvil 7 días)')

ax2.set_ylabel('Engagement score', color='red', fontsize=12)
ax2.tick_params(axis='y', labelcolor='red')

# --- Título y estética ---
plt.title('Evolución del precio de Dogecoin vs engagement de tweets de Elon Musk relacionados con la criptomoneda',
          fontsize=16, fontweight='bold', pad=20)

fig.tight_layout()

# Leyendas combinadas
lines_labels = [ax.get_legend_handles_labels() for ax in [ax1, ax2]]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
ax1.legend(lines, labels, loc='upper center', frameon=True, fontsize=11, facecolor='white', edgecolor='gray')

plt.show()

In [ ]:
# 4. Agrupación semanal para mostrar cómo evoluciona el engagement semanal frente al precio semanal

# Agrupar semanalmente promediando valores numéricos
df_weekly = df_merged.set_index('Date').resample('W').mean(numeric_only=True).reset_index()

# Calcular correlación semanal
correlacion_semanal = df_weekly[['engagement_score', 'daily_return']].corr().iloc[0, 1]
print(f"📅 Correlación semanal entre engagement y retorno: {correlacion_semanal:.4f}")

In [ ]:
# 5. Vamos a comprobar si el engagement de la semana anterior predice el retorno de la semana actual.
# Esto es muy útil para evaluar si el engagement anticipa movimientos de precio.
# Para ello aplicamos un desfase (lag).

# Crear una nueva columna con el engagement de la semana anterior
df_weekly['engagement_lag1'] = df_weekly['engagement_score'].shift(1)

# Calcular correlación entre engagement lag y retorno actual
correlacion_lag1 = df_weekly[['engagement_lag1', 'daily_return']].corr().iloc[0, 1]
print(f"📊 Correlación con desfase 1 semana: {correlacion_lag1:.4f}")

In [ ]:
# 6. Calcular variación de precio (día siguiente)
df_merged['next_day_price'] = df_merged['Close'].shift(-1)
df_merged['price_change'] = df_merged['next_day_price'] - df_merged['Close']

# Correlación
corr = df_merged[['engagement_score', 'price_change']].corr()
print(corr)

In [ ]:
# 7. Calcular variación de precio a 2 días después
df_merged['price_change_2d'] = df_merged['Close'].shift(-2) - df_merged['Close']
print(df_merged[['engagement_score', 'price_change_2d']].corr())

In [ ]:
# 8. A veces, tweets con mucho engagement aumentan la volatilidad, no necesariamente la dirección:
df = df_merged.copy()

# Asegurar orden temporal
df = df.sort_values("Date").reset_index(drop=True)

# Volatilidad: rango intradía (High-Low) / Close
df["volatility"] = (df["High"] - df["Low"]) / df["Close"]

# Seleccionar columnas de interés para el cálculo de la correlación
corr_df = df[['engagement_score', 'volatility', 'Volume']]

# Correlación de Pearson
corr_matrix = corr_df.corr()
print(corr_matrix)